# [모듈 1.1] CodeCommit을 하면 EventBridge를 통해 CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 'SageMaker Pipeline config 파일(sm_pipeline_train_config.json)'을 수정 후 push 하면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다.

## 참고 자료
- Amazon EventBridge 유저 가이드
   (https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-what-is.html)
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)



# 1. 환경 확인

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')

%store -r s3_input_data_uri
print("s3_input_data_uri: n", s3_input_data_uri)


s3_input_data_uri: n s3://sagemaker-ap-northeast-2-057716757052/NCFModel/data2


# 2. 리파지토리 로컬에 클로닝

### repository_url :
```
repository_url =  "https://git-codecommit.<REGION>.amazonaws.com/v1/repos/ncf-training-repo"
```

In [2]:
## Codecommit Repo Name (ncf-training-repo)
repository_name = "ncf-training-repo"


## 4_sm-train-codepipeline에서 만든 훈련 용 repository name 로딩

#%store -r training_repository_name
#repository_name = training_repository_name

print(repository_name)

ncf-training-repo


In [3]:
repository_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(repository_url)

# repository_url = "<Repository URL>"

https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo


In [4]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

Cloning into 'ncf-training-repo'...
remote: Counting objects: 47, done.        


# 3. 파라미터 세팅

In [5]:

# code pipeline name, arn, role_arn
codepipeline_name1 = "ncf-code-pipeline-training-sm-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name1).get('metadata').get('pipelineArn')
eventbrideexcrole_arn = boto3.client('iam').get_role(RoleName = 'MLOps-EventBridgeRole').get('Role').get('Arn')

# EventBridge Rule Name
eventbridge_rule_name1 = "codepipelinerule1"

# Training data path (Prefix)
data_prefix = "NCFModel/data2"
training_files = 'codecommit1'

# 4. EventBridge Rule 정의

## 4.1 EventPattern 정의

In [6]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

arn:aws:codecommit:ap-northeast-2:057716757052:ncf-training-repo


#### 위 CodeCommit arn 출력값을 복사해서 아래 resources 부분에 붙여넣으세요.

In [7]:
eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name1,
    EventPattern = """{
        "source": ["aws.codecommit"],
        "detail-type": ["CodeCommit Repository State Change"],
        "resources": ["arn:aws:codecommit:ap-northeast-2:057716757052:ncf-training-repo"],
        "detail": {
            "referenceType": ["branch"],
            "referenceName": ["master"]
        }
    }"""
)


In [8]:
eventresponse

{'RuleArn': 'arn:aws:events:ap-northeast-2:057716757052:rule/codepipelinerule1',
 'ResponseMetadata': {'RequestId': '84fffc1e-5aa1-475e-a0fb-2987b752322b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84fffc1e-5aa1-475e-a0fb-2987b752322b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Wed, 21 Dec 2022 06:47:39 GMT'},
  'RetryAttempts': 0}}

## 4.2 Event Target 설정

In [9]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name1,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name1,
            'RoleArn' : eventbrideexcrole_arn
        }
    ]
)


In [10]:
eventresponse

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': '9530fbea-b86d-441f-a8bc-8a92e7f7deba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9530fbea-b86d-441f-a8bc-8a92e7f7deba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Wed, 21 Dec 2022 06:47:41 GMT'},
  'RetryAttempts': 0}}

# 5. 새로운 SageMaker Pipeline (Training) 구성 파일 준비

## 5.1. 변수 설정 및 dict 로 저장

In [11]:
project_prefix = "SageMaker-Train-NCF"
bucket = sagemaker.Session().default_bucket()
region = sagemaker.Session().boto_region_name

train_sm_pipeline_name = "ncf-training-code-pipeline-sm-pipeline"
ModelApprovalStatus="PendingManualApproval"
inference_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"
training_instance_type = "ml.p3.2xlarge"
training_instance_count = 1

sm_pipeline_train_config_json_path = f'src/sm_pipeline_train_config.json'


In [12]:
from datetime import datetime
currentDateAndTime = datetime.now()

# 현재 시간을 Json 에 넣기 위함
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
print("The current time is", currentTime)

The current time is 2022-12-21-06-47-44


In [13]:
sm_train_pipeline_dict = {
    "project_prefix" : project_prefix,            
    "s3_input_data_uri" : s3_input_data_uri,
    "sm_pipeline_name" : train_sm_pipeline_name,
    "training_instance_type" :  training_instance_type,    
    "training_instance_count" :  training_instance_count,        
    "ModelApprovalStatus" :  ModelApprovalStatus,    
    "inference_image_uri" :  inference_image_uri,
    "bucket" : bucket,        
    "update_time" :  currentTime    
}

## 5.2. 설정 딕션너리 Json 파일로 저장 후 테스트를 위해 로딩

In [14]:
import json
from common_utils import save_json, load_json

save_json(sm_pipeline_train_config_json_path, sm_train_pipeline_dict)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)
print (json.dumps(sm_pipeline_train_dict, indent=2))

src/sm_pipeline_train_config.json is saved
{
  "project_prefix": "SageMaker-Train-NCF",
  "s3_input_data_uri": "s3://sagemaker-ap-northeast-2-057716757052/NCFModel/data2",
  "sm_pipeline_name": "ncf-training-code-pipeline-sm-pipeline",
  "training_instance_type": "ml.p3.2xlarge",
  "training_instance_count": 1,
  "ModelApprovalStatus": "PendingManualApproval",
  "inference_image_uri": "763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:1.8.1-gpu-py3",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "update_time": "2022-12-21-06-47-44"
}


## 5.3 Config 파일 복사

### 5.3.1 codecommit1 폴더 생성

In [15]:
newdir = training_files+"/pipelines/ncf/src/"
! rm -rf $newdir
! mkdir -p $newdir

### 5.3.2 sm_pipeline_train_config_json 파일 Copy

In [16]:
source_path = sm_pipeline_train_config_json_path
target_path = newdir

! cp {source_path} {target_path}

# 6. SageMaker Pipeline config 파일을 로컬 레파지토리 디렉토리에 복사

In [17]:
%%sh -s {repository_name} {training_files}
repository_name=$1
training_files=$2

cp -r $training_files/* $repository_name
ls $repository_name

codebuild-buildspec.yml
CONTRIBUTING.md
img
LICENSE
origin-sagemaker-pipelines-project.ipynb
pipelines
pushtime.log
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py


# 7. 리파지토리에 푸시 (이벤트 트리거)

In [18]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
date >> pushtime.log
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/6_sm-eventbridge/ncf-training-repo
[master 79ea8b7] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-26-229.ap-northeast-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 2 insertions(+), 1 deletion(-)


remote: Validating objects: 100%        
To https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo
   9de4acf..79ea8b7  master -> master


# 8. 변수 저장

In [19]:
%store eventbridge_rule_name1
%store codepipeline_name1

Stored 'eventbridge_rule_name1' (str)
Stored 'codepipeline_name1' (str)


# 9. EventBridge Rule 및 CodePipeline 실행 확인 (콘솔 화면)

## 9.1 EventBridge Rule 생성 확인
#### 하단에 codepipeline1이 생성된 것을 확인할 수 있습니다.

![eventbridge1.png](img/eventbridge1.png)

## 9.2 codepipelinerule1 의 Event pattern 확인

![eventbridge1-eventpattern.png](img/eventbridge1-eventpattern.png)

## 9.3 codepipelinerule1의 Target 확인

![eventbridge1-target.png](img/eventbridge1-target.png)

## 9.4 EventBridge를 통한 CodePipeline 실행 확인

![codepipeline-training-inprogress1.png](img/codepipeline-training-inprogress1.png)